In [1]:
from import_all import *
import visa
import time
import datetime
VISA_PATH = '/Library/Frameworks/Visa.framework/VISA'


In [3]:
rm = visa.ResourceManager(VISA_PATH)
inst = rm.open_resource('GPIB0::13::INSTR')
info = inst.query('*IDN?')
print info


KEITHLEY INSTRUMENTS INC.,MODEL 2400,1386470,C32   Oct  4 2010 14:20:11/A02  /S/K



In [73]:
def integrate_current(int_time, volt, v_rang=10, c_cmpl=0.1, c_rang=0.0001):
    """integrate current for the designated time
    
    Arg:
    int_time: float, time in ms.
    """
    
    inpt = inst.write('*RST')
    inpt = inst.write(':SOUR:FUNC VOLT')
    inpt = inst.write(':SOUR:VOLT:MODE FIXED')
    inpt = inst.write(':SOUR:VOLT:RANG ' + str(v_rang))
    inpt = inst.write(':SOUR:VOLT:LEV ' + str(volt))
    inpt = inst.write(':SENS:CURR:PROT ' + str(c_cmpl)) 
    inpt = inst.write(':SENS:FUNC "CURR"')
    inpt = inst.write(':SENS:CURR:RANG ' + str(c_rang))
    inpt = inst.write(':FORM:ELEM CURR')

    if int_time < 166:
        nplc = str(int_time/(1./60)/1000)
        
        inpt = inst.write(':SENS:NPLC ' + nplc)
        inpt = inst.write(':OUTP ON')
        output = inst.query(':READ?')
        inpt = inst.write('OUTP OFF')
        
        print 'no cycle'
        return float(output)
        
    else:
        cycle = int(int_time/100) + 1

        inpt = inst.write(':SENS:NPLC ' + '6') #100 ms per cycle    
        inpt = inst.write(':OUTP ON')

        current_list = []
        for c in range(cycle):
            now = datetime.datetime.now()
            elasped = (now - now0).total_seconds()
            output = inst.query(':READ?')
            current_list.append(float(output))

        inpt = inst.write('OUTP OFF')

        #omit first value as it is always lower than the others
        c_list_np = np.array(current_list[1:])
        mean_current = c_list_np.mean()
        return mean_current, c_list_np
    
integrate_current(1, 4)

no cycle


1.022181e-09

In [17]:
a = 32.244/0.1
print int(a)

322


In [18]:
print float('-8.185930E-08')

-8.18593e-08
